In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time

# CSV 파일 읽기
df = pd.read_csv("dc_(ID,Artist).csv")

# 결과를 저장할 리스트
results = []

# 첫 번째 아티스트에 대해 데이터 수집
for index, row in df.iterrows():  # 첫 번째 행만 선택
    artist_name = row["Artist"]
    artist_id = row["dcinside_ID"]  # dcinside_ID 사용
    print(f'수집 중: 아티스트 "{artist_name}" (ID: {artist_id})')

    page = 1  # 페이지 초기화

    scrapying = True

    while scrapying:  # 페이지를 반복하며 수집
        url = f"https://gall.dcinside.com/board/lists/?id={artist_id}&page={page}"
        print(url)

        # 웹 페이지 요청 (User-Agent 추가)
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
        }

        max_retries = 3
        for attempt in range(max_retries):
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                break
            print(f"요청 실패, 재시도 {attempt + 1}/{max_retries} - 상태 코드: {response.status_code}")
            time.sleep(2)  # 재시도 전 대기

        # 응답이 성공적일 경우
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            posts = soup.find_all("tr", class_="ub-content us-post")  # 게시글 목록
            
            # 게시글이 없다면 종료
            if not posts:
                print(f"    더 이상 게시글이 없습니다. 아티스트: {artist_name}, 페이지: {page}. 수집 종료.")
                break
            
            for post in posts:
                # 공지 여부 확인
                if post.get("data-type") == "icon_notice":
                    print("    공지 게시글 발견: 건너뜁니다.")
                    continue

                # 작성일 추출
                post_date = post.find("td", class_="gall_date").text.strip()

                # 월.일 형식인지 확인하고 연도 추가
                if len(post_date) == 5:  # "MM.DD" 형식인지 확인
                    formatted_post_date = f"24.{post_date}"  # "24.MM.DD" 형식으로 변환
                else:
                    formatted_post_date = post_date  # 기존 형식 사용

                # 기준 날짜 설정
                cutoff_date = "23.09.23"
                formatted_cutoff_date = "23.09.23"

                # 문자열을 datetime 객체로 변환
                post_date_dt = datetime.strptime(formatted_post_date, "%y.%m.%d")
                cutoff_date_dt = datetime.strptime(formatted_cutoff_date, "%y.%m.%d")

                # 날짜 비교
                if post_date_dt <= cutoff_date_dt:
                    print(f"    종료 조건 충족: 작성일 {formatted_post_date}. 수집 완료.")
                    scrapying = False
                    break
                
                # 조회수 및 추천수 추출
                views = post.find("td", class_="gall_count").text.strip()
                recommendations = post.find("td", class_="gall_recommend").text.strip()
                
                # 결과 리스트에 추가
                results.append({
                    "아티스트": artist_name,
                    "dcinside_ID": artist_id,
                    "작성일": formatted_post_date,
                    "조회수": views,
                    "추천수": recommendations
                })
                print(f"    수집됨: 작성일 {formatted_post_date}, 조회수 {views}, 추천수 {recommendations}")

            # 페이지 증가
            page += 1
            print(f"    다음 페이지: {page}")
            time.sleep(3)  # 요청 간에 3초 대기
        else:
            print(f"{artist_name}의 데이터 가져오기 오류: {response.status_code}")
            break

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# 결과를 CSV 파일로 저장
results_df.to_csv("output.csv", index=False)
print("데이터 수집 완료. 결과를 'output.csv'로 저장했습니다.")


수집 중: 아티스트 "AKMU (악뮤)" (ID: akdong)
https://gall.dcinside.com/board/lists/?id=akdong&page=1
    공지 게시글 발견: 건너뜁니다.
    공지 게시글 발견: 건너뜁니다.
    공지 게시글 발견: 건너뜁니다.
    공지 게시글 발견: 건너뜁니다.
    수집됨: 작성일 24.09.28, 조회수 20, 추천수 0
    수집됨: 작성일 24.09.23, 조회수 41, 추천수 0
    수집됨: 작성일 24.09.04, 조회수 54, 추천수 0
    수집됨: 작성일 24.07.18, 조회수 176, 추천수 0
    수집됨: 작성일 24.07.14, 조회수 158, 추천수 0
    수집됨: 작성일 24.07.10, 조회수 135, 추천수 0
    수집됨: 작성일 24.06.28, 조회수 193, 추천수 0
    수집됨: 작성일 24.06.27, 조회수 290, 추천수 0
    수집됨: 작성일 24.06.26, 조회수 313, 추천수 0
    수집됨: 작성일 24.06.17, 조회수 183, 추천수 0
    수집됨: 작성일 24.06.16, 조회수 587, 추천수 0
    수집됨: 작성일 24.06.15, 조회수 471, 추천수 0
    수집됨: 작성일 24.06.07, 조회수 964, 추천수 0
    수집됨: 작성일 24.06.03, 조회수 498, 추천수 0
    수집됨: 작성일 24.05.22, 조회수 344, 추천수 0
    수집됨: 작성일 24.05.15, 조회수 334, 추천수 1
    수집됨: 작성일 24.04.19, 조회수 563, 추천수 0
    수집됨: 작성일 24.04.16, 조회수 544, 추천수 0
    수집됨: 작성일 24.04.14, 조회수 371, 추천수 0
    수집됨: 작성일 24.04.02, 조회수 1077, 추천수 0
    수집됨: 작성일 24.03.31, 조회수 480, 추천수 0
    수집됨: 작성일 24.03.30, 조

KeyboardInterrupt: 

In [7]:
print(results_df)

Empty DataFrame
Columns: []
Index: []
